# Preprocessing

In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [2]:
import torch

In [3]:
from src.vfm.connection import Connection
from src.vfm.preprocessor import Preprocessor
import pandas as pd
from src.vfm.model.reg.gradient_boost import GradientBoost
from src.vfm.model.nn.swt_tft import SWTTFTModel
from src.utils.descriptive_analysis import *
from src.vfm.model.physics.physics_informed import PhysicsInformedHybridModel, PhysicsModel
import matplotlib.pyplot as plt
import os

c:\Users\perer\Documents\Code\pcperera\vfm\.venv\Lib\site-packages\pytorch_forecasting\models\base\_base_model.py:28: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [4]:
connection = Connection()
# Get client
client = connection.get_client()

In [5]:
well = f"W06"
df_raw = connection.get_data(client, well=well)


In [6]:
df_raw.columns


Index(['whp', 'wht', 'dhp', 'dht', 'choke', 'dcp', 'qo_well_test',
       'qg_well_test', 'qw_well_test', 'qo_mpfm', 'qg_mpfm', 'wc_mpfm',
       'well_id'],
      dtype='object')

In [7]:
dependent_vars=["qo_well_test", "qg_well_test", "qw_well_test"]
independent_vars=["dhp", "dht", "whp", "wht", "choke", "dcp"]

In [50]:
summarize_na(df_raw)

whp             240
wht             240
dhp             240
dht             240
choke           240
dcp             240
qo_well_test    138
qg_well_test    137
qw_well_test    236
qo_mpfm         240
qg_mpfm         240
wc_mpfm         240
well_id           0
dtype: int64

In [198]:
oldest_ts_per_column = df_raw.notna().apply(lambda col: col.index[col].min())
oldest_ts_per_column

whp            2017-03-06 02:38:30
wht            2017-03-06 02:38:30
dhp            2017-03-06 02:38:30
dht            2017-03-06 02:38:30
choke          2017-03-06 02:38:30
dcp            2017-03-06 02:38:30
qo_well_test   2019-01-30 20:39:00
qg_well_test   2019-01-30 20:39:00
qw_well_test   2019-01-30 20:39:00
qo_mpfm        2017-03-06 02:38:30
qg_mpfm        2017-03-06 02:38:30
wc_mpfm        2017-03-06 02:38:30
well_id        2017-03-06 02:38:30
dtype: datetime64[ns]

In [199]:
latest_ts_per_column = df_raw.notna().iloc[::-1].idxmax()
latest_ts_per_column

whp            2023-05-07 07:24:00
wht            2023-05-07 07:24:00
dhp            2023-05-07 07:24:00
dht            2023-05-07 07:24:00
choke          2023-05-07 07:24:00
dcp            2023-05-07 07:24:00
qo_well_test   2023-01-17 20:24:00
qg_well_test   2023-01-17 20:24:00
qw_well_test   2023-01-17 20:24:00
qo_mpfm        2023-05-07 07:24:00
qg_mpfm        2023-05-07 07:24:00
wc_mpfm        2023-05-07 07:24:00
well_id        2023-05-07 07:24:00
dtype: datetime64[ns]

In [200]:
for var in independent_vars:
    print(f"{var.upper()}:")
    print_values(df=df_raw, column=var)

DHP:
(Timestamp('2017-03-06 02:38:30'), 140.80273689090015)
(Timestamp('2017-03-06 10:21:00'), 145.24109692686594)
(Timestamp('2017-04-09 06:46:30'), 132.6083139460912)
(Timestamp('2017-04-12 18:15:30'), 132.57501060912443)
(Timestamp('2017-04-13 21:09:30'), 131.86284085723275)
(Timestamp('2017-04-16 00:44:30'), 129.8963902756642)
(Timestamp('2017-05-25 21:52:00'), 133.87188200484064)
(Timestamp('2017-05-27 18:24:00'), 132.89176087636628)
(Timestamp('2017-06-07 01:46:00'), 131.4358744091226)
(Timestamp('2017-06-09 03:55:00'), 131.52888809972296)
(Timestamp('2017-06-11 15:20:30'), 130.26530450012373)
(Timestamp('2017-06-14 09:34:30'), 129.45389476772456)
(Timestamp('2017-11-03 22:27:30'), 134.44115935287982)
(Timestamp('2017-11-06 08:42:30'), 134.8615459666893)
(Timestamp('2018-01-09 06:23:00'), 144.68535587667074)
(Timestamp('2018-01-11 14:24:30'), 142.5874481155357)
(Timestamp('2018-01-14 01:46:00'), 146.08301377898556)
(Timestamp('2018-05-27 13:23:00'), 154.30044337225846)
(Timestamp

In [201]:
df_raw = df_raw.loc["2017-03-06":"2021-04-05"]

In [202]:
for var in independent_vars:
    print(f"{var.upper()}:")
    print_values(df=df_raw, column=var)

DHP:
(Timestamp('2017-03-06 02:38:30'), 140.80273689090015)
(Timestamp('2017-03-06 10:21:00'), 145.24109692686594)
(Timestamp('2017-04-09 06:46:30'), 132.6083139460912)
(Timestamp('2017-04-12 18:15:30'), 132.57501060912443)
(Timestamp('2017-04-13 21:09:30'), 131.86284085723275)
(Timestamp('2017-04-16 00:44:30'), 129.8963902756642)
(Timestamp('2017-05-25 21:52:00'), 133.87188200484064)
(Timestamp('2017-05-27 18:24:00'), 132.89176087636628)
(Timestamp('2017-06-07 01:46:00'), 131.4358744091226)
(Timestamp('2017-06-09 03:55:00'), 131.52888809972296)
(Timestamp('2017-06-11 15:20:30'), 130.26530450012373)
(Timestamp('2017-06-14 09:34:30'), 129.45389476772456)
(Timestamp('2017-11-03 22:27:30'), 134.44115935287982)
(Timestamp('2017-11-06 08:42:30'), 134.8615459666893)
(Timestamp('2018-01-09 06:23:00'), 144.68535587667074)
(Timestamp('2018-01-11 14:24:30'), 142.5874481155357)
(Timestamp('2018-01-14 01:46:00'), 146.08301377898556)
(Timestamp('2018-05-27 13:23:00'), 154.30044337225846)
(Timestamp

In [203]:
preprocessor = Preprocessor(df=df_raw)
df = preprocessor.preprocess_timeseries(well_id=well)
df.shape

(17886, 11)

In [204]:
# Count NaNs per column
summarize_na(df)

dhp                 0
dht                 0
whp                 0
wht                 0
dcp                 0
choke               0
well_id             0
qo_well_test    17751
qg_well_test    17750
qw_well_test    17813
time_idx            0
dtype: int64

In [205]:
df.to_csv(r"metrics\df.csv", index=True)

In [206]:
df[independent_vars].describe()

,dhp,dht,whp,wht,choke,dcp
count,17886.000000,17886.000000,17886.000000,17886.000000,17886.000000,17886.000000
mean,146.208258,81.720296,44.142979,61.902286,0.583714,21.951055
std,8.444019,0.230292,6.003180,1.509886,0.187241,0.442830
min,129.453895,79.946527,32.335916,55.487664,0.255000,20.957746
25%,141.548548,81.706564,40.810640,61.240703,0.454000,21.600806
50%,146.629300,81.777256,44.050034,61.995001,0.553122,22.009258
75%,152.074585,81.805195,48.498756,62.875298,0.664000,22.231987
max,163.951129,81.917253,56.594563,64.859727,0.957951,23.140249


In [207]:
df[dependent_vars].describe()

,qo_well_test,qg_well_test,qw_well_test
count,135.000000,136.000000,73.000000
mean,115.959196,14089.091681,5.406620
std,37.432924,4778.315138,4.440104
min,1.182995,33.976968,0.000000
25%,97.450042,11623.268919,2.742500
50%,129.630833,15724.561697,4.991333
75%,143.451495,17765.340198,7.398244
max,165.836056,20036.177302,19.396149


In [208]:
summarize_na(df)

dhp                 0
dht                 0
whp                 0
wht                 0
dcp                 0
choke               0
well_id             0
qo_well_test    17751
qg_well_test    17750
qw_well_test    17813
time_idx            0
dtype: int64

In [209]:
df_with_targets = df.dropna(subset=["qo_well_test", "qg_well_test", "qw_well_test"])
df_with_targets.shape

(73, 11)

In [210]:
summarize_na(df_with_targets)

dhp             0
dht             0
whp             0
wht             0
dcp             0
choke           0
well_id         0
qo_well_test    0
qg_well_test    0
qw_well_test    0
time_idx        0
dtype: int64

In [211]:
df_with_targets.to_csv(r"metrics\df_with_targets.csv", index=True)

# Gradient Boost

In [212]:
gb_model = GradientBoost(dependent_vars=dependent_vars, independent_vars=independent_vars, df_target=df_with_targets)

In [213]:
gb_model.train()

qo_well_test surrogate model trained, R2 score: 0.640
qg_well_test surrogate model trained, R2 score: 0.633
qw_well_test surrogate model trained, R2 score: -2.372


In [214]:
# df_dense = gb_model.generate_dense_well_rates(df=df)

# Physics Informed Hybrid Model

In [215]:
# Fit hybrid model
hybrid_model = PhysicsInformedHybridModel(independent_vars=independent_vars, dependant_vars=dependent_vars)
hybrid_model.fit(df_with_targets)
print("Pysics R2:", hybrid_model.physics_score(df_with_targets))
print("Physics Informed Hybrid R2:", hybrid_model.score(df_with_targets))


Pysics R2: {'r2_qo': 0.3931562509403439, 'r2_qw': 0.32388529552180423, 'r2_qg': 0.21870763492744238}
Physics Informed Hybrid R2: {'r2_qo': 0.9873465294751299, 'r2_qw': 0.9956751295524446, 'r2_qg': 1.0}


In [216]:
df_complete = hybrid_model.generate_dense_well_rates(df=df)

In [217]:
df_complete

,dhp,dht,whp,wht,dcp,choke,well_id,qo_well_test,qg_well_test,qw_well_test,time_idx
2017-03-06 02:00:00,140.802737,81.667259,41.730152,61.559255,21.704721,0.528083,W06,116.284470,14926.028338,13.438316,0
2017-03-06 04:00:00,141.912327,81.669240,42.523851,61.542837,21.656272,0.528083,W06,113.936844,14834.098201,14.044753,1
2017-03-06 06:00:00,143.021917,81.671221,43.317550,61.526418,21.607823,0.528083,W06,112.183645,14817.138917,13.914168,2
2017-03-06 08:00:00,144.131507,81.673202,44.111249,61.509999,21.559375,0.528083,W06,110.726332,14837.397893,13.611290,3
2017-03-06 10:00:00,145.241097,81.675182,44.904948,61.493580,21.510926,0.443929,W06,128.045340,13256.268898,13.033353,4
...,...,...,...,...,...,...,...,...,...,...,...
2021-04-04 04:00:00,153.046547,81.892279,49.146725,58.155375,21.448616,0.440699,W06,86.622960,10952.607780,10.378882,17881
2021-04-04 06:00:00,153.046547,81.892279,49.146725,58.155375,21.448616,0.440699,W06,91.718595,10952.607783,10.989425,17882
2021-04-04 08:00:00,153.046547,81.892279,49.146725,58.155375,21.448616,0.440699,W06,86.622960,10952.607780,10.378882,17883
2021-04-04 10:00:00,153.046547,81.892279,49.146725,58.155375,21.448616,0.440699,W06,86.622960,10952.607780,10.378882,17884


In [218]:
summarize_na(df_complete)

dhp             0
dht             0
whp             0
wht             0
dcp             0
choke           0
well_id         0
qo_well_test    0
qg_well_test    0
qw_well_test    0
time_idx        0
dtype: int64

# Sparse Well-Test Temporal Fusion Transformer Model - Well 06

In [224]:
swt_tft_model = SWTTFTModel(dependent_variables=dependent_vars,
                            independent_variables=independent_vars,
                             df=df_complete)

c:\Users\perer\Documents\Code\pcperera\vfm\.venv\Lib\site-packages\lightning\pytorch\utilities\parsing.py:210: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
c:\Users\perer\Documents\Code\pcperera\vfm\.venv\Lib\site-packages\lightning\pytorch\utilities\parsing.py:210: Attribute 'logging_metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['logging_metrics'])`.
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores


In [ ]:
swt_tft_model.train()

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name                               | Type                            | Params | Mode  | FLOPs
--------------------------------------------------------------------------------------------------------
0  | loss                               | MultiLoss                       | 0      | train | 0    
1  | logging_metrics                    | ModuleList                      | 0      | train | 0    
2  | input_embeddings                   | MultiEmbedding                  | 1      | train | 0    
3  | prescalers                         | ModuleDict                      | 480    | train | 0    
4  | static_variable_selection          | VariableSelectionNetwork        | 14.0 K | train | 0    
5  | encoder_variable_selection         | VariableSelectionNetwork        | 16.1 K | train | 0    
6  | decoder_variable_selection         | VariableSelectionNetwork        | 3.8 K  | train | 0    
7  | static_context_variable_selection  | GatedResidualNetwo

<class 'pytorch_forecasting.models.temporal_fusion_transformer._tft.TemporalFusionTransformer'>
Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

c:\Users\perer\Documents\Code\pcperera\vfm\.venv\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:434: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=21` in the `DataLoader` to improve performance.


c:\Users\perer\Documents\Code\pcperera\vfm\.venv\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:434: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=21` in the `DataLoader` to improve performance.


Epoch 1: 100%|██████████| 4471/4471 [09:20<00:00,  7.97it/s, v_num=8, train_loss_step=524.0, val_loss=476.0, train_loss_epoch=757.0]  

In [ ]:
import matplotlib.pyplot as plt

for well_idx in range(1):  # Plot all workload timeseries
    # for field_idx in range(len(target_fields)):
        fig, ax = plt.subplots(figsize=(10, 4))
        best_tft.plot_prediction(x=raw_predictions.x, out=raw_predictions.output, idx=well_idx, add_loss_to_title=False, ax=ax)
        current_title = ax.get_title()
        new_title = f' {well} qg_mpfm - known reals {time_varying_known_reals} \n' + current_title
        ax.set_title(new_title)